In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA

# 데이터 로드
df = pd.read_csv('Mall_Customers.csv')

In [ ]:
# 1. 데이터 탐색
df.info()
df.isnull().sum()
df.describe()

In [3]:
# 2. 데이터 전처리
# 수치형 컬럼 선택
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns


In [4]:
# 스케일링
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[numeric_columns])
scaled_data = pd.DataFrame(scaled_features, columns=numeric_columns)

In [5]:
# 3. 최적의 클러스터 수 찾기 (엘보우 메소드 & 실루엣 분석)
max_k = 10
inertias = []
silhouette_scores = []

for k in range(2, max_k+1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_data)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(scaled_data, kmeans.labels_))

In [ ]:
# 엘보우 & 실루엣 그래프
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(2, max_k+1), inertias, marker='o')
plt.xlabel('클러스터 수')
plt.ylabel('Inertia')
plt.title('엘보우 메소드')

plt.subplot(1, 2, 2)
plt.plot(range(2, max_k+1), silhouette_scores, marker='o')
plt.xlabel('클러스터 수')
plt.ylabel('실루엣 점수')
plt.title('실루엣 분석')

plt.tight_layout()
plt.show()

In [7]:
# 4. K-means 클러스터링 (예시로 5개 클러스터 사용)
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_labels = kmeans.fit_predict(scaled_data)

In [ ]:
# K-means 결과 시각화
pca = PCA(n_components=2)
data_2d = pca.fit_transform(scaled_data)

plt.figure(figsize=(10, 6))
scatter = plt.scatter(data_2d[:, 0], data_2d[:, 1], c=kmeans_labels, cmap='viridis')
plt.colorbar(scatter)
plt.title('K-means 클러스터링 결과')
plt.xlabel('첫 번째 주성분')
plt.ylabel('두 번째 주성분')
plt.show()

In [ ]:
# 5. 계층적 군집화
linkage_matrix = linkage(scaled_data, method='ward')

plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix)
plt.title('계층적 군집화 덴드로그램')
plt.xlabel('샘플 인덱스')
plt.ylabel('거리')
plt.show()

In [10]:
# 6. DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(scaled_data)

In [ ]:
# DBSCAN 결과 시각화
plt.figure(figsize=(10, 6))
scatter = plt.scatter(data_2d[:, 0], data_2d[:, 1], c=dbscan_labels, cmap='viridis')
plt.colorbar(scatter)
plt.title('DBSCAN 클러스터링 결과')
plt.xlabel('첫 번째 주성분')
plt.ylabel('두 번째 주성분')
plt.show()


In [ ]:

# 7. 클러스터링 결과 분석
print("\nK-means 클러스터별 샘플 수:")
print(pd.Series(kmeans_labels).value_counts().sort_index())

print("\nDBSCAN 클러스터별 샘플 수:")
print(pd.Series(dbscan_labels).value_counts().sort_index())